# 02 – Preprocesamiento de texto para detección de toxicidad

En este notebook vamos a construir el pipeline de **preprocesamiento de texto** sobre el dataset
`youtoxic_english_1000.csv` para dejarlo listo de cara a modelos **clásicos** (TF-IDF + clasificadores).

Concretamente:

1. Cargaremos el dataset original.
2. Definiremos una función de limpieza básica:
   - Conversión de emojis a texto en inglés.
   - Paso a minúsculas.
   - Eliminación de URLs, menciones, hashtags, saltos de línea y espacios extra.
3. Implementaremos un pipeline clásico de NLP en inglés:
   - Tokenización.
   - Eliminación de stopwords.
   - Lematización.
4. Crearemos:
   - `text_basic`: texto con limpieza ligera (apto tanto para modelos clásicos como modernos).
   - `text_classic`: texto procesado (tokenizado, sin stopwords, lematizado) para modelos clásicos.
5. Guardaremos un CSV preprocesado que se usará en el notebook de modelado.

> Nota: la vectorización (TF-IDF, Bag-of-Words, etc.) se hará en el notebook de modelado,
> a partir de la columna `text_classic`.

## Paso 1: Importar librerías y configurar el entorno

Importamos las librerías necesarias para el preprocesamiento y definimos las rutas del fichero de entrada y de salida.


In [1]:
# Paso 1: Import libraries for preprocessing

import pandas as pd                      # Data manipulation with DataFrames
import numpy as np                       # Numerical utilities (if needed)
import re                                # Regular expressions for text cleaning and tokenization
import emoji                             # Emoji handling: convert emojis to text
import nltk                              # Classic NLP library: stopwords, lemmatization, etc.
from nltk.corpus import stopwords        # Stopword lists for multiple languages (we use English)
from nltk.stem import WordNetLemmatizer  # Lemmatizer based on WordNet (English)
from IPython.display import Markdown, display  # Display Markdown explanations from code cells

# Paths for input (original dataset) and output (preprocessed dataset)
INPUT_PATH = "../../data/youtoxic_english_1000.csv"
OUTPUT_PATH = "../../data/youtoxic_english_1000_clean.csv"

display(Markdown(
"""
**Salida:**

Esta celda importa las librerías necesarias para el preprocesamiento y define las rutas
del fichero de entrada y de salida. Todavía no se ha cargado ni transformado el dataset.
"""
))



**Salida:**

Esta celda importa las librerías necesarias para el preprocesamiento y define las rutas
del fichero de entrada y de salida. Todavía no se ha cargado ni transformado el dataset.


## Paso 2: Descargar recursos de NLTK (si es necesario)

NLTK necesita descargar algunos recursos la primera vez:

- `stopwords` → lista de stopwords en inglés.
- `wordnet`, `omw-1.4` → recursos para lematización.

Esta celda se ejecuta normalmente una sola vez por entorno.


In [2]:
# Paso 2: Download required NLTK resources (run once per environment)

nltk.download("stopwords")   # English stopwords list
nltk.download("wordnet")     # WordNet lexical database for lemmatization
nltk.download("omw-1.4")     # Additional WordNet data

display(Markdown(
"""
**Explicación de la salida:**

Esta celda descarga (si es necesario) los recursos de NLTK que se usarán para:

- Obtener la lista de stopwords en inglés.
- Lematizar palabras mediante WordNet.

Si ya estaban descargados, NLTK los reutiliza y la descarga será muy rápida.
"""
))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alfbb\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alfbb\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\alfbb\AppData\Roaming\nltk_data...



**Explicación de la salida:**

Esta celda descarga (si es necesario) los recursos de NLTK que se usarán para:

- Obtener la lista de stopwords en inglés.
- Lematizar palabras mediante WordNet.

Si ya estaban descargados, NLTK los reutiliza y la descarga será muy rápida.


## Paso 3: Configurar stopwords y lematizador

En esta celda:

- Creamos el conjunto de stopwords en inglés.
- Instanciamos el lematizador de WordNet.


In [3]:
# Paso 3: Configure English stopwords and lemmatizer

# English stopwords set (e.g. "the", "is", "at", "which", "on", ...)
stopwords_en = set(stopwords.words("english"))

# WordNet-based lemmatizer for English
lemmatizer = WordNetLemmatizer()

display(Markdown(
"""
**Explicación de la salida:**

- `stopwords_en` contiene la lista de stopwords en inglés, que utilizaremos para
  eliminar palabras muy frecuentes y poco informativas.
- `lemmatizer` nos permitirá obtener el **lema** (forma base) de cada palabra
  en inglés (por ejemplo, *running* → *run*).

Estas herramientas se usarán dentro del pipeline de preprocesamiento clásico.
"""
))



**Explicación de la salida:**

- `stopwords_en` contiene la lista de stopwords en inglés, que utilizaremos para
  eliminar palabras muy frecuentes y poco informativas.
- `lemmatizer` nos permitirá obtener el **lema** (forma base) de cada palabra
  en inglés (por ejemplo, *running* → *run*).

Estas herramientas se usarán dentro del pipeline de preprocesamiento clásico.


## Paso 4: Cargar el dataset original

Vamos a cargar el CSV original con comentarios y etiquetas, tal y como viene de origen.


In [4]:
# Paso 4: Load the original dataset

df = pd.read_csv(INPUT_PATH)  # Load CSV with comments and labels

display(df.head())
print(f"\nNúmero de filas: {df.shape[0]}")
print(f"Número de columnas: {df.shape[1]}")

display(Markdown(
"""
**Explicación de la salida:**

- Se muestran las primeras filas del dataset original.
- Se confirma el número de filas y columnas.

Este será el punto de partida para aplicar el preprocesamiento de texto.
"""
))


,CommentId,VideoId,Text,IsToxic,IsAbusive,IsThreat,IsProvocative,IsObscene,IsHatespeech,IsRacist,IsNationalist,IsSexist,IsHomophobic,IsReligiousHate,IsRadicalism
0,Ugg2KwwX0V8-aXgCoAEC,04kJtp6pVXI,If only people would just take a step back and...,False,False,False,False,False,False,False,False,False,False,False,False
1,Ugg2s5AzSPioEXgCoAEC,04kJtp6pVXI,Law enforcement is not trained to shoot to app...,True,True,False,False,False,False,False,False,False,False,False,False
2,Ugg3dWTOxryFfHgCoAEC,04kJtp6pVXI,\r\nDont you reckon them 'black lives matter' ...,True,True,False,False,True,False,False,False,False,False,False,False
3,Ugg7Gd006w1MPngCoAEC,04kJtp6pVXI,There are a very large number of people who do...,False,False,False,False,False,False,False,False,False,False,False,False
4,Ugg8FfTbbNF8IngCoAEC,04kJtp6pVXI,"The Arab dude is absolutely right, he should h...",False,False,False,False,False,False,False,False,False,False,False,False



Número de filas: 1000
Número de columnas: 15



**Explicación de la salida:**

- Se muestran las primeras filas del dataset original.
- Se confirma el número de filas y columnas.

Este será el punto de partida para aplicar el preprocesamiento de texto.


## Paso 5: Definir limpieza básica de texto (incluyendo emojis)

Definiremos una función `basic_clean` que haga:

1. Conversión de emojis a texto en inglés (`emoji.demojize`).
2. Paso a minúsculas.
3. Eliminación de:
   - URLs.
   - Menciones (`@usuario`) y hashtags (`#tema`).
   - Saltos de línea.
4. Normalización de espacios (evitar espacios múltiples).

El resultado será la columna `text_basic`, que es una versión del texto con ruido reducido
y que puede ser común tanto para modelos clásicos como para modelos modernos.


In [5]:
# Paso 5: Define a basic text cleaning function (including emojis)

# Regular expression patterns for cleaning
url_pattern = r"http\S+|www\S+"            # URLs (http://, https://, www.)
mention_hashtag_pattern = r"(@\w+)|(#\w+)" # Mentions (@user) and hashtags (#topic)
newline_pattern = r"[\r\n]+"               # Newline characters

def basic_clean(text: str) -> str:
    """
    Apply basic cleaning to a text comment:
    1. Convert emojis to text (in English).
    2. Convert to lowercase.
    3. Remove URLs.
    4. Remove mentions and hashtags.
    5. Remove newlines.
    6. Normalize multiple spaces.
    """
    # Ensure we work with a string
    if not isinstance(text, str):
        text = str(text)

    # 1) Convert emojis to text, e.g. "I ❤️ you" -> "I :red_heart: you"
    text = emoji.demojize(text, language="en")

    # 2) Convert to lowercase
    text = text.lower()

    # 3) Remove URLs
    text = re.sub(url_pattern, " ", text)

    # 4) Remove mentions and hashtags
    text = re.sub(mention_hashtag_pattern, " ", text)

    # 5) Remove newline characters
    text = re.sub(newline_pattern, " ", text)

    # 6) Normalize whitespace (collapse multiple spaces into one)
    text = re.sub(r"\s+", " ", text).strip()

    return text

display(Markdown(
"""
**Explicación de la salida:**

No se muestra nada todavía, pero hemos definido `basic_clean`, una función que:

- Convierte emojis a texto (en inglés), preservando la información emocional.
- Normaliza el texto a minúsculas.
- Elimina URLs, menciones, hashtags y saltos de línea.
- Limpia espacios extra.

Esta limpieza es relativamente ligera y puede usarse tanto antes de modelos clásicos como modernos.
"""
))



**Explicación de la salida:**

No se muestra nada todavía, pero hemos definido `basic_clean`, una función que:

- Convierte emojis a texto (en inglés), preservando la información emocional.
- Normaliza el texto a minúsculas.
- Elimina URLs, menciones, hashtags y saltos de línea.
- Limpia espacios extra.

Esta limpieza es relativamente ligera y puede usarse tanto antes de modelos clásicos como modernos.


## Paso 6: Tokenización, eliminación de stopwords y lematización (pipeline clásico)

Ahora definiremos funciones para el preprocesamiento clásico:

1. `tokenize(text)`  
   - Convierte un string en una lista de tokens (palabras) usando una expresión regular simple.
2. `remove_stopwords(tokens)`  
   - Elimina stopwords en inglés.
3. `lemmatize_tokens(tokens)`  
   - Aplica lematización para reducir cada palabra a su forma base.

Estas funciones serán específicas para el pipeline de **modelos clásicos**.


In [6]:
# Paso 6: Define tokenization, stopword removal, and lemmatization functions (classic NLP pipeline)

def tokenize(text: str) -> list[str]:
    """
    Tokenize text into a list of word tokens using a simple regex.
    Only alphabetic characters and apostrophes are kept (English-focused).
    """
    tokens = re.findall(r"[a-zA-Z']+", text)
    return tokens

def remove_stopwords(tokens: list[str]) -> list[str]:
    """
    Remove English stopwords from the list of tokens.
    """
    filtered_tokens = [t for t in tokens if t not in stopwords_en]
    return filtered_tokens

def lemmatize_tokens(tokens: list[str]) -> list[str]:
    """
    Lemmatize tokens using WordNet lemmatizer.
    (Simple version: no POS tagging, assumes default part-of-speech.)
    """
    lemmatized = [lemmatizer.lemmatize(t) for t in tokens]
    return lemmatized

display(Markdown(
"""
**Explicación de la salida:**

Hemos definido tres funciones clave para el pipeline clásico:

1. `tokenize(text)`  
   - Tokeniza el texto en una lista de palabras usando una expresión regular orientada a inglés.

2. `remove_stopwords(tokens)`  
   - Elimina las stopwords en inglés (the, is, at, which, on, ...).

3. `lemmatize_tokens(tokens)`  
   - Aplica lematización para convertir cada palabra a su forma base usando WordNet.

Estas funciones se encadenarán en un pipeline que generará un texto preparado
para modelos clásicos (TF-IDF, BoW, etc.).
"""
))



**Explicación de la salida:**

Hemos definido tres funciones clave para el pipeline clásico:

1. `tokenize(text)`  
   - Tokeniza el texto en una lista de palabras usando una expresión regular orientada a inglés.

2. `remove_stopwords(tokens)`  
   - Elimina las stopwords en inglés (the, is, at, which, on, ...).

3. `lemmatize_tokens(tokens)`  
   - Aplica lematización para convertir cada palabra a su forma base usando WordNet.

Estas funciones se encadenarán en un pipeline que generará un texto preparado
para modelos clásicos (TF-IDF, BoW, etc.).


## Paso 7: Definir el pipeline completo para modelos clásicos

Crearemos una función `preprocess_text_classic` que aplique, en este orden:

1. `basic_clean` → limpieza ligera (emojis, minúsculas, URLs, menciones...).
2. `tokenize` → transformar texto en lista de palabras.
3. `remove_stopwords` → eliminar palabras poco informativas.
4. `lemmatize_tokens` → normalizar palabras a su lema.
5. Unir los tokens resultantes en un string final (`"token1 token2 ..."`).

Esta función generará la columna `text_classic`, pensada para vectorización clásica.


In [7]:
# Paso 7: Define full preprocessing pipeline function for classic models

def preprocess_text_classic(text: str) -> str:
    """
    Apply the full preprocessing pipeline for classic models:
    1. Basic cleaning (emojis -> text, lowercase, URLs/mentions removal, etc.).
    2. Tokenization.
    3. English stopword removal.
    4. Lemmatization.
    5. Join processed tokens into a single string.
    """
    # 1) Basic cleaning
    cleaned = basic_clean(text)

    # 2) Tokenization
    tokens = tokenize(cleaned)

    # 3) Stopword removal
    tokens_no_stop = remove_stopwords(tokens)

    # 4) Lemmatization
    tokens_lemma = lemmatize_tokens(tokens_no_stop)

    # 5) Join tokens back into a single string
    processed_text = " ".join(tokens_lemma)

    return processed_text

display(Markdown(
"""
**Explicación de la salida:**

La función `preprocess_text_classic` encadena todo el pipeline clásico:

1. Limpieza básica (`basic_clean`).
2. Tokenización (`tokenize`).
3. Eliminación de stopwords (`remove_stopwords`).
4. Lematización (`lemmatize_tokens`).
5. Unión de tokens en un texto procesado.

Este texto procesado es ideal para vectorizadores como `CountVectorizer` o `TfidfVectorizer`
en el notebook de modelado.
"""
))



**Explicación de la salida:**

La función `preprocess_text_classic` encadena todo el pipeline clásico:

1. Limpieza básica (`basic_clean`).
2. Tokenización (`tokenize`).
3. Eliminación de stopwords (`remove_stopwords`).
4. Lematización (`lemmatize_tokens`).
5. Unión de tokens en un texto procesado.

Este texto procesado es ideal para vectorizadores como `CountVectorizer` o `TfidfVectorizer`
en el notebook de modelado.


## Paso 8: Aplicar el preprocesamiento al dataset

Vamos a crear dos columnas:

- `text_basic` → resultado de aplicar `basic_clean` (limpieza ligera).
- `text_classic` → resultado de aplicar `preprocess_text_classic` (pipeline clásico completo).

Luego mostraremos algunos ejemplos comparando:

- `Text` (original).
- `text_basic`.
- `text_classic`.


In [8]:
# Paso 8: Apply preprocessing to the whole dataset

# Light-cleaned text (shared baseline)
df["text_basic"] = df["Text"].apply(basic_clean)

# Fully processed text for classic models
df["text_classic"] = df["Text"].apply(preprocess_text_classic)

# Show a few random examples to inspect the transformations
examples = df[["Text", "text_basic", "text_classic"]].sample(5, random_state=42)
display(examples)

display(Markdown(
"""
**Explicación de la salida:**

La tabla muestra, para varios comentarios:

- `Text`: texto original sin preprocesar.
- `text_basic`: texto tras la limpieza básica (emojis a texto, minúsculas, sin URLs ni menciones).
- `text_classic`: texto tras el pipeline clásico completo (limpieza básica + tokenización + stopwords + lemas).

Esto permite comprobar visualmente que el preprocesamiento es razonable y que el texto
sigue siendo interpretable después de las transformaciones.
"""
))


,Text,text_basic,text_classic
521,You call yourself an anarchist but defend a co...,you call yourself an anarchist but defend a co...,call anarchist defend cop shooting unarmed civ...
737,My mother told me the same thing. God Bless t...,my mother told me the same thing. god bless th...,mother told thing god bless woman
740,Love it I same the saem thing Go Peggy! #stup...,love it i same the saem thing go peggy! ya kil...,love saem thing go peggy ya killing ya self qu...
660,"Next time they do that, line up some cars and ...","next time they do that, line up some cars and ...",next time line car start making burnout smoke ...
411,He was Robbing the Store and Being a Big Man ....,he was robbing the store and being a big man ....,robbing store big man play fire get burnt poli...



**Explicación de la salida:**

La tabla muestra, para varios comentarios:

- `Text`: texto original sin preprocesar.
- `text_basic`: texto tras la limpieza básica (emojis a texto, minúsculas, sin URLs ni menciones).
- `text_classic`: texto tras el pipeline clásico completo (limpieza básica + tokenización + stopwords + lemas).

Esto permite comprobar visualmente que el preprocesamiento es razonable y que el texto
sigue siendo interpretable después de las transformaciones.


## Paso 9 (opcional): Longitud del texto preprocesado

Podemos añadir:

- `text_len_classic`: longitud en caracteres de `text_classic`.
- `word_count_classic`: número de palabras en `text_classic`.

Estas columnas pueden servir como features adicionales o para análisis posterior.


In [9]:
# Paso 9: Compute length metrics on processed text (optional)

df["text_len_classic"] = df["text_classic"].str.len()
df["word_count_classic"] = df["text_classic"].str.split().str.len()

display(df[["text_classic", "text_len_classic", "word_count_classic"]].head())

display(Markdown(
"""
**Explicación de la salida:**

- `text_len_classic`: número de caracteres del texto ya preprocesado.
- `word_count_classic`: número de palabras del texto preprocesado.

Estas columnas ayudan a entender cómo el pipeline de preprocesamiento ha afectado
a la longitud de los comentarios y pueden utilizarse como variables adicionales
en los modelos si se considera útil.
"""
))


,text_classic,text_len_classic,word_count_classic
0,people would take step back make case anyone e...,840,127
1,law enforcement trained shoot apprehend traine...,90,13
2,dont reckon 'black life matter' banner held wh...,256,40
3,large number people like police officer called...,339,49
4,arab dude absolutely right shot extra time sho...,135,22



**Explicación de la salida:**

- `text_len_classic`: número de caracteres del texto ya preprocesado.
- `word_count_classic`: número de palabras del texto preprocesado.

Estas columnas ayudan a entender cómo el pipeline de preprocesamiento ha afectado
a la longitud de los comentarios y pueden utilizarse como variables adicionales
en los modelos si se considera útil.


## Paso 10: Guardar el dataset preprocesado

Guardaremos un nuevo CSV con:

- Columnas originales.
- Columnas de preprocesamiento:
  - `text_basic`
  - `text_classic`
  - `text_len_classic`
  - `word_count_classic`

Este fichero será el punto de partida en el notebook de modelado.


In [10]:
# Paso 10: Save preprocessed dataset to CSV

df.to_csv(OUTPUT_PATH, index=False)
print(f"Dataset preprocesado guardado en: {OUTPUT_PATH}")

display(Markdown(
f"""
**Explicación de la salida:**

Se ha guardado el dataset preprocesado en:

`{OUTPUT_PATH}`

El fichero contiene:
- Todas las columnas originales (texto, IDs y etiquetas `Is...`).
- `text_basic`: texto con limpieza ligera (útil para pipelines clásicos y modernos).
- `text_classic`: texto preprocesado para modelos clásicos (tokenización + stopwords + lemas).
- `text_len_classic` y `word_count_classic`: métricas de longitud del texto procesado.

El siguiente paso será usar este fichero en el notebook de **modelado**,
donde aplicaremos vectorización (TF-IDF, BoW, etc.) y entrenaremos modelos de clasificación.
"""
))


Dataset preprocesado guardado en: ../../data/youtoxic_english_1000_clean.csv



**Explicación de la salida:**

Se ha guardado el dataset preprocesado en:

`../../data/youtoxic_english_1000_clean.csv`

El fichero contiene:
- Todas las columnas originales (texto, IDs y etiquetas `Is...`).
- `text_basic`: texto con limpieza ligera (útil para pipelines clásicos y modernos).
- `text_classic`: texto preprocesado para modelos clásicos (tokenización + stopwords + lemas).
- `text_len_classic` y `word_count_classic`: métricas de longitud del texto procesado.

El siguiente paso será usar este fichero en el notebook de **modelado**,
donde aplicaremos vectorización (TF-IDF, BoW, etc.) y entrenaremos modelos de clasificación.


# Resumen del notebook de preprocesamiento

En este notebook hemos:

1. Cargado el dataset original de comentarios de YouTube.
2. Definido una función de **limpieza básica** (`basic_clean`) que:
   - Convierte emojis a texto en inglés.
   - Pasa a minúsculas.
   - Elimina URLs, menciones, hashtags y saltos de línea.
   - Normaliza espacios.
3. Configurado un pipeline clásico de NLP en inglés:
   - `tokenize` → tokeniza el texto.
   - `remove_stopwords` → elimina stopwords en inglés.
   - `lemmatize_tokens` → lematiza las palabras usando WordNet.
4. Implementado `preprocess_text_classic` para encadenar todos los pasos y obtener:
   - `text_classic`: texto limpio, tokenizado, sin stopwords y lematizado.
5. Creado una versión de limpieza ligera (`text_basic`) que puede servir también como entrada
   para modelos modernos si en el futuro se usan transformers.
6. Añadido columnas de longitud sobre el texto procesado.
7. Guardado un CSV preprocesado (`youtoxic_english_1000_clean.csv`) listo para el notebook de modelado.

---

## Próximos pasos

En el notebook de **modelado** (por ejemplo, `4.3-modeling-baseline.ipynb`):

1. Cargar `youtoxic_english_1000_clean.csv`.
2. Elegir la etiqueta objetivo (p. ej. `IsToxic`).
3. Hacer un **train/test split**.
4. Vectorizar `text_classic` con:
   - `CountVectorizer` o `TfidfVectorizer`.
5. Entrenar modelos clásicos:
   - Logistic Regression, Linear SVM, Naive Bayes, etc.
6. Calcular métricas (accuracy, precision, recall, F1, matriz de confusión).
7. Comparar modelos y, a partir de ahí, avanzar a ensembles, tuning o incluso modelos modernos.

Con estos dos notebooks (4.2-eda y 4.2-preprocessing) ya tienes la base sólida de datos listos para modelar. 💪
